In [3]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
import pymongo
import math
import pandas as pd
from multiprocessing.dummy import Pool
from multiprocessing import cpu_count

In [4]:
mc = pymongo.MongoClient()
db = mc['reports']
mt_ellinor_reports = db['mt_ellinor']

 WARNING: If you want to clear the current database

In [ ]:
mt_ellinor_reports.drop()

If you need to problem solve: %pdb

In [5]:
def select_text(parent_element, css_selector):
    element = parent_element.select_one(css_selector)
    return get_text_if_not_none(element)

In [6]:
def get_text_if_not_none(element):
    if element is None:
        text = None
    else:
        text = element.text
    return text

In [7]:
def parse_trip_report(trip_report_div):
    """Return a dictionary representing a single trip report."""
    creator = select_text(trip_report_div, 'div.CreatorInfo span a')
    date = select_text(trip_report_div, 'span.elapsed-time')
    report = select_text(trip_report_div,'div.show-with-full')
    trail_conditions = select_text(trip_report_div,'div.trail-issues')
    votes = select_text(trip_report_div,'span.UpvoteCount')
    return {
        "Creator": creator,
        "Date": date,
        "Report": report,
        "Trail_condtions": trail_conditions,
        "Votes": votes
    }

In [8]:
def get_trail_report(title, hikeurl, params=None):
    """Accepts a url of the hike, finds all of the subsequent trip reports for that hike,
    scrapes them and inserts them into a MongoDB
    **Input parameters**
    ------------------------------------------------------------------------------
    title: string.  Hike name.
    hikeurl: string. Base URL for the request.
    params: dictionary.  Parameters to be included in the request.
    **Output**
    ------------------------------------------------------------------------------
    None. Appends entry to MongoDB using Pymongo
    """
    r = requests.get(hikeurl + '/@@related_tripreport_listing', params).text
    #
    soup = BeautifulSoup(r, 'lxml')
    for trip_report_div in soup.select('div#trip-reports div.item'):
        trip_report = parse_trip_report(trip_report_div)
        mt_ellinor_reports.insert_one(trip_report)

In [9]:
def iterate_all_reports(title, hikeurl):
    """Determines the number of times to call getTripReports function based on
    the number of trip reports listed on the hike homepage.
    **Input parameters**
    ------------------------------------------------------------------------------
    title: string.  Hike name.
    hikeurl: string. Base URL for the request.
    **Output**
    ------------------------------------------------------------------------------
    None. Appends entry to MongoDB using pymongo.
    """
    #lists how many reports are on the page
    r = requests.get(hikeurl + '/@@related_tripreport_listing').text
    soup = BeautifulSoup(r, 'lxml')
    numit = math.ceil(float(soup.find('div', {'id': 'count-data'}).text)/5)
    for i in range(int(numit)):
        get_trail_report(title, hikeurl, params={'b_start:int': str(i*5)})

In [10]:
iterate_all_reports('mt_ellinor',"https://www.wta.org/go-hiking/hikes/mount-ellinor")

TypeError: string indices must be integers

In [ ]:
df = pd.DataFrame(list( mt_ellinor_reports.find()))